# Rerun RGBD camera example
This example shows how to use [rerun](https://www.rerun.io/) for logging images, image annotations, pointclouds, transforms between elements in the world, time-series in a way that you might want to do for a robotic manipulation setup. Make sure you have a **ZED2I camera connected**.


Rerun has more features such as logging meshes, logging 3D bboxes, URDFs (in process). Check the docs to learn more. 
Also note that this tool is still very 'young', it became public only mid february 2023. There are many issues that need to be solved. The team is very responsive on github so shoot if you find issues etc.

In [ ]:
from airo_camera_toolkit.cameras.zed2i import Zed2i
import rerun as rr
#autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# start rerun. If the UI is already running, it will connect to it. Otherwise it will start a new UI and connect to it.
# you can also start rerun using `python -m rerun`.
rr.init("test",spawn=True)

In [ ]:
zed = Zed2i(depth_mode=Zed2i.NEURAL_DEPTH_MODE, resolution=Zed2i.RESOLUTION_2K)

In [ ]:
pointcloud = zed.get_colored_point_cloud()
rgb = zed.get_rgb_image()
depth = zed.get_depth_image()

In [ ]:
# log the colored pointcloud to the UI
rr.log("world/camera1/pointcloud", rr.Points3D(positions=pointcloud[:,:3],colors=pointcloud[:,3:6]))


In [ ]:
pointcloud[3156][3:6]

In [ ]:
# log the non-colored pointcloud to the UI in the same world/camera1 space
rr.log("world/camera1/non_color_pointcloud", rr.Points3D(positions=pointcloud[:,:3],colors=[0.5,0.5,0.5]))

In [ ]:
# log the rgb and depth images to the UI in a new image/camera1 space, this will automatically become a 2D image viewer.
rr.log("image/camera1/rgb",rr.Image(rgb))
rr.log("image/camera1/depth",rr.Image(depth))

In [ ]:
# log a dummy 2D bbox 
rr.log("image/camera1/rect", rr.Boxes2D(mins=[20,100], sizes=[300,500]))

In [ ]:
# log some dummy keypoints and attach labels
rr.log("image/camera1/keypoints", rr.Points2D([[600,500],[400,500]],keypoint_ids=[0,1],radii=20, labels=["keypoint1","keypoint2"]))

In [ ]:
# log dummy camera extrinsics from the world space. 
# This specifies the pose of camera in world.
translation = [0,0.5,0.7] 
rotation = [-0.707,0,0,0.707] #scalar-last! 
# rr.log_rigid3("world/camera1", parent_from_child=(translation, rotation))
rr.log("world/camera1", rr.Transform3D(translation=translation, rotation=rotation))

In [ ]:
# log the actual camera intrinsics, to create a pinhole camera model in the UI.
rr.log("world/camera1/rgb", rr.Pinhole(image_from_camera=zed.intrinsics_matrix(), resolution=zed.resolution))
rr.log("world/camera1/rgb",rr.Image(rgb))

In [ ]:
# set up the 'view' of the 3D world space. This is for convenience so that rerun can sensible starting orientations for the spaces. 
rr.log("world", rr.ViewCoordinates.RIGHT_HAND_Z_UP, timeless=True)
rr.log("world/camera1", rr.ViewCoordinates.RDF, timeless=True)

In [ ]:
# log some more data
for _ in range(5):
    pointcloud = zed.get_colored_point_cloud()
    rr.log("world/camera1/pointcloud", rr.Points3D(positions=pointcloud[:,:3],colors=pointcloud[:,3:6]))

In [ ]:
# log time-series (e.g. F/T sensor)
for i in range(100):
    rr.log("world/robot/force",rr.TimeSeriesScalar(i))


In [ ]:
# close the camera
zed.camera.close()